# referenced from Akira's git, code to download the data directly from NYC website
from os.path import getsize
from urllib.request import urlretrieve

output_dir = "../raw_data"
fname_templates = "yellow_tripdata_2018"

for m in range(1, 13):
    month = str(m).zfill(2)
    out = f'{fname_template}-{month}.csv'
    url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/{out}"
    urlretrieve(url, f"{output_dir}/{out}")

    print(f"Done downloading {out} to {output_dir} with size {getsize(f'{output_dir}/{out}') / 1073741824:.2f}GB")

# referenced from Akira's git, code to download the data directly from NYC website
from os.path import getsize
from urllib.request import urlretrieve

output_dir = "../raw_data"
fname_template = "yellow_tripdata_2019"

for m in range(1, 13):
    month = str(m).zfill(2)
    out = f'{fname_template}-{month}.csv'
    url = f"https://s3.amazonaws.com/nyc-tlc/trip+data/{out}"
    urlretrieve(url, f"{output_dir}/{out}")

    print(f"Done downloading {out} to {output_dir} with size {getsize(f'{output_dir}/{out}') / 1073741824:.2f}GB")

In [8]:
#unzipping the taxi zones, NTA and census files
import zipfile

directory = "../raw_data_lite/"
taxi_zones = f"{directory}taxi_zones.zip"
nta = f"{directory}nynta_21b.zip"
census_data = f"{directory}ACSDP5Y2018.DP03_2021-08-03T091618.zip"

zipfiles = (taxi_zones, nta, census_data)
for file in zipfiles:
    with zipfile.ZipFile(file) as z:
        # make sure they are in the same directory
        z.extractall("../raw_data_lite/")
        

In [15]:

import geopandas as gpd
#converting the NTA and taxi zones data into geojson files
taxi_zones_shp = gpd.read_file(f"{directory}taxi_zones.shp")
taxi_zones_shp.to_file(f"{directory}taxi_zones.geojson", drivers = 'GeoJSON')

nta_shp = gpd.read_file(f"{directory}nynta_21b/nynta.shp")
nta_shp.to_file(f"{directory}nta.geojson", drivers = 'GeoJSON')
#reading the files in as dataframes
taxi_zones = gpd.read_file(f"{directory}taxi_zones.geojson")
nta = gpd.read_file(f"{directory}nta.geojson")

#merging the taxi zones and the NTA into a new dataframe
zones = gpd.sjoin(taxi_zones, nta, how = 'inner')
zones

,OBJECTID,Shape_Leng_left,Shape_Area_left,zone,LocationID,borough,geometry,index_right,BoroCode,BoroName,CountyFIPS,NTACode,NTAName,Shape_Leng_right,Shape_Area_right
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",181,3,Brooklyn,047,BK99,park-cemetery-etc-Brooklyn,385538.602814,2.253406e+08
10,11,0.079211,0.000265,Bath Beach,11,Brooklyn,"POLYGON ((983945.076 158978.174, 984160.158 15...",181,3,Brooklyn,047,BK99,park-cemetery-etc-Brooklyn,385538.602814,2.253406e+08
13,14,0.175214,0.001382,Bay Ridge,14,Brooklyn,"POLYGON ((974794.238 174019.157, 975135.579 17...",181,3,Brooklyn,047,BK99,park-cemetery-etc-Brooklyn,385538.602814,2.253406e+08
25,26,0.123548,0.000534,Borough Park,26,Brooklyn,"POLYGON ((988880.135 172984.800, 988969.535 17...",181,3,Brooklyn,047,BK99,park-cemetery-etc-Brooklyn,385538.602814,2.253406e+08
33,34,0.065706,0.000174,Brooklyn Navy Yard,34,Brooklyn,"MULTIPOLYGON (((991923.998 197615.718, 992381....",181,3,Brooklyn,047,BK99,park-cemetery-etc-Brooklyn,385538.602814,2.253406e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,245,0.095983,0.000466,West Brighton,245,Staten Island,"POLYGON ((957085.564 172591.260, 957142.385 17...",110,5,Staten Island,085,SI22,West New Brighton-New Brighton-St. George,66054.251409,5.603045e+07
250,251,0.137711,0.000626,Westerleigh,251,Staten Island,"POLYGON ((947868.004 169247.734, 948000.981 16...",110,5,Staten Island,085,SI22,West New Brighton-New Brighton-St. George,66054.251409,5.603045e+07
199,200,0.112662,0.000745,Riverdale/North Riverdale/Fieldston,200,Bronx,"POLYGON ((1012885.034 268830.484, 1012900.378 ...",100,2,Bronx,005,BX22,North Riverdale-Fieldston-Riverdale,31851.142476,4.904828e+07
219,220,0.106016,0.000313,Spuyten Duyvil/Kingsbridge,220,Bronx,"POLYGON ((1012377.010 263347.798, 1012114.369 ...",100,2,Bronx,005,BX22,North Riverdale-Fieldston-Riverdale,31851.142476,4.904828e+07


In [16]:
# reference from Akira's git
#forming a schema for the dataframes
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col

ints = ('VendorID', 'passenger_count', 'RateCodeID', 'RatecodeID','payment_type', 'PULocationID', 'DOLocationID')
doubles = ('trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount'
          , 'congestion_surcharge')
strings = ('store_and_fwd_flag',)
dtimes = ('tpep_pickup_datetime', 'tpep_dropoff_datetime')


dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})
print(dtypes)

{'VendorID': IntegerType, 'passenger_count': IntegerType, 'RateCodeID': IntegerType, 'RatecodeID': IntegerType, 'payment_type': IntegerType, 'PULocationID': IntegerType, 'DOLocationID': IntegerType, 'trip_distance': DoubleType, 'fare_amount': DoubleType, 'extra': DoubleType, 'mta_tax': DoubleType, 'tip_amount': DoubleType, 'tolls_amount': DoubleType, 'improvement_surcharge': DoubleType, 'total_amount': DoubleType, 'congestion_surcharge': DoubleType, 'store_and_fwd_flag': StringType, 'tpep_pickup_datetime': TimestampType, 'tpep_dropoff_datetime': TimestampType}


In [12]:
# Importing and starting a spark session
from pyspark.sql import SparkSession
from pyspark import SparkContext
#Supress warnings
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()
sc.setLogLevel('WARN')

spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)
#Make the spark files present well
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

21/08/04 00:04:57 WARN Utils: Your hostname, LAPTOP-D5HGLKLK resolves to a loopback address: 127.0.1.1; using 172.24.145.93 instead (on interface eth0)
21/08/04 00:04:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/04 00:04:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


# reference from Akira's git
#forming a schema for the dataframes
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import col

ints = ('VendorID', 'passenger_count', 'RateCodeID', 'RatecodeID','payment_type', 'PULocationID', 'DOLocationID')
doubles = ('trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount'
          , 'congestion_surcharge')
strings = ('store_and_fwd_flag',)
dtimes = ('tpep_pickup_datetime', 'tpep_dropoff_datetime')


dtypes = {column: IntegerType() for column in ints}
dtypes.update({column: DoubleType() for column in doubles})
dtypes.update({column: StringType() for column in strings})
dtypes.update({column: TimestampType() for column in dtimes})


In [19]:
taxi_dir = "../raw_data/yellow_tripdata_20"

taxi18_01 = spark.read.csv(f"{taxi_dir}18-01.csv", header=True)
taxi18_02 = spark.read.csv(f"{taxi_dir}18-02.csv", header=True)
taxi18_03 = spark.read.csv(f"{taxi_dir}18-03.csv", header=True)
taxi18_04 = spark.read.csv(f"{taxi_dir}18-04.csv", header=True)
taxi18_05 = spark.read.csv(f"{taxi_dir}18-05.csv", header=True)
taxi18_06 = spark.read.csv(f"{taxi_dir}18-06.csv", header=True)
taxi18_07 = spark.read.csv(f"{taxi_dir}18-07.csv", header=True)
taxi18_08 = spark.read.csv(f"{taxi_dir}18-08.csv", header=True)
taxi18_09 = spark.read.csv(f"{taxi_dir}18-09.csv", header=True)
taxi18_10 = spark.read.csv(f"{taxi_dir}18-10.csv", header=True)
taxi18_11 = spark.read.csv(f"{taxi_dir}18-11.csv", header=True)
taxi18_12 = spark.read.csv(f"{taxi_dir}18-12.csv", header=True)

taxi19_01 = spark.read.csv(f"{taxi_dir}19-01.csv", header=True)
taxi19_02 = spark.read.csv(f"{taxi_dir}19-02.csv", header=True)
taxi19_03 = spark.read.csv(f"{taxi_dir}19-03.csv", header=True)
taxi19_04 = spark.read.csv(f"{taxi_dir}19-04.csv", header=True)
taxi19_05 = spark.read.csv(f"{taxi_dir}19-05.csv", header=True)
taxi19_06 = spark.read.csv(f"{taxi_dir}19-06.csv", header=True)
taxi19_07 = spark.read.csv(f"{taxi_dir}19-07.csv", header=True)
taxi19_08 = spark.read.csv(f"{taxi_dir}19-08.csv", header=True)
taxi19_09 = spark.read.csv(f"{taxi_dir}19-09.csv", header=True)
taxi19_10 = spark.read.csv(f"{taxi_dir}19-10.csv", header=True)
taxi19_11 = spark.read.csv(f"{taxi_dir}19-11.csv", header=True)
taxi19_12 = spark.read.csv(f"{taxi_dir}19-12.csv", header=True)




In [21]:
taxi19_01.limit(5)

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.50,1,N,151,239,1,7,0.5,0.5,1.65,0,0.3,9.95,null
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.60,1,N,239,246,1,14,0.5,0.5,1,0,0.3,16.3,null
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,.00,1,N,236,236,1,4.5,0.5,0.5,0,0,0.3,5.8,null
2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,.00,1,N,193,193,2,3.5,0.5,0.5,0,0,0.3,7.55,null
2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,.00,2,N,193,193,2,52,0,0.5,0,0,0.3,55.55,null
